In [1]:
import logging
logging.basicConfig(
    level=logging.INFO,
    format='[%(levelname)s] %(name)s - %(message)s'
)
sensitive_attributes = {'sex' : {'columns' : [
                                            {
                                            "name": "sex",
                                            "underprivileged": [2]
                                            }
                                        ],
                             'type' : 'simple'},
                        'ethnicity' : {'columns' : [
                                                                {
                                                                "name": "ethnicity",
                                                                "privileged": [1]
                                                                }
                                                            ],
                                                'type' : 'simple'},
                        'age' : {'columns' : [
                                            {
                                            "name": "age",
                                            "privileged": [3,4]
                                            }
                                        ],
                             'type' : 'simple'},
                        'sex_ethnicity' : {'groups' : ["sex","ethnicity"],
                                                        'type' : 'complex'}}


from eticas.model.ml_model import MLModel
model = MLModel(
    model_name="ML Testing Regression",
    description="A logistic regression model to illustrate audits",
    country="USA",
    state="CA",
    sensitive_attributes=sensitive_attributes,
    features=["feature_0", "feature_1", "feature_2"]
)


model.run_training_audit(dataset_path ='files/example_training_binary_2.csv', 
                                                label_column = 'outcome', output_column = 'predicted_outcome',positive_output = [1])
model.run_production_audit(dataset_path ='files/example_operational_binary_2.csv', 
                                                output_column = 'predicted_outcome',positive_output = [1])
model.run_impacted_audit(dataset_path ='files/example_impact_binary_2.csv', 
                                                output_column = 'recorded_outcome',positive_output = [1])
model.run_drift_audit(dataset_path_dev = 'files/example_training_binary_2.csv', 
                      output_column_dev = 'outcome', positive_output_dev = [1],
                    dataset_path_prod = 'files/example_operational_binary_2.csv', 
                      output_column_prod = 'predicted_outcome', positive_output_prod = [1])
audit_result = model.df_results(norm_values=True)

[INFO] eticas.model.ml_model - Running training audit for model: ML Testing Regression
[INFO] eticas.audit.labeled_audit - Running training audit on model 'ML Testing Regression'
[INFO] eticas.data.loaders - Dataset successfully loaded from files/example_training_binary_2.csv (CSV).
[INFO] eticas.audit.labeled_audit - Training data loaded '(10000, 14)'
[INFO] eticas.metrics.da_inconsistency - Completed: 'Da_inconsistency'
[INFO] eticas.metrics.da_positive - Completed: 'Da_positive'
[INFO] eticas.metrics.dxa_inconsistency - Completed: 'Dxa_inconsistency'
[INFO] eticas.metrics.da_informative - Completed: 'Da_informative'
[INFO] eticas.metrics.d_statisticalparity - Completed: 'D_statisticalparity'
[INFO] eticas.metrics.d_parity - Completed: 'D_parity'
[INFO] eticas.metrics.performance - Completed: 'Performance'
[INFO] eticas.metrics.d_equalodds - Completed: 'D_equalodds'
[INFO] eticas.metrics.d_calibrated - Completed: 'D_calibrated'
[INFO] eticas.model.ml_model - training audit finished f

In [2]:
import pandas as pd
pd.set_option('display.max_rows', 500)

audit_result

value
group        metric              attribute     stage                   
benchmarking da_inconsistency    age           01-training         44.8
                                               02-production       44.5
                                               03-impact           45.0
                                 ethnicity     01-training         40.0
                                               02-production       20.0
                                               03-impact           10.0
                                 sex           01-training         60.0
                                               02-production       30.0
                                               03-impact           15.0
                                 sex_ethnicity 01-training         25.0
                                               02-production       15.0
                                               03-impact           10.0
             da_positive         age           01-training         45.2
                                               02-production       44.9
                                               03-impact           45.3
                                 ethnicity     01-training         39.8
                                               02-production       15.8
                                               03-impact            3.7
                                 sex           01-training         59.8
                                               02-production       19.7
                                               03-impact            5.7
                                 sex_ethnicity 01-training         24.7
                                               02-production       10.0
                                               03-impact            3.7
distribution da_informative      age           01-training         98.0
                                               02-production        100
                                               03-impact            100
                                 ethnicity     01-training        99.72
                                               02-production        100
                                               03-impact        72.3467
                                 sex           01-training        99.08
                                               02-production    90.9812
                                               03-impact        60.8824
                                 sex_ethnicity 01-training          100
                                               02-production    90.9812
                                               03-impact        60.8824
             drift               age           -                    100
                                 ethnicity     -                 49.995
                                 overall       -                 48.975
                                 sex           -                 49.995
                                 sex_ethnicity -                 49.995
             dxa_inconsistency   age           01-training        99.28
                                               02-production     98.944
                                               03-impact         98.736
                                 ethnicity     01-training        99.64
                                               02-production     89.776
                                               03-impact        77.8347
                                 sex           01-training       99.128
                                               02-production    77.3333
                                               03-impact        70.1013
                                 sex_ethnicity 01-training       99.656
                                               02-production     86.264
                                               03-impact        77.8347
fairness     d_calibrated_false  age           training           99.76
                                 e

In [6]:
audit_result['age']['fairness']

{'ref': 80,
 'training_DI': 98.0,
 'training_SPD': 98.0,
 'training_TPR': 96.82,
 'training_FPR': 99.76,
 'training_PPV': 96.82,
 'training_PNV': 99.76,
 'impact_DI': 99.0,
 'impact_SPD': 98.0}

In [4]:
model.drift_results

{'tdx_inconsistency': {'overall': {'accuracy': 0.6735,
   'proportion': 0.5,
   'rate': 1.347,
   'normalized_risk': 48.975,
   'bias_level': 'High bias'},
  'sex': {'accuracy': 0.6667,
   'proportion': 0.5,
   'rate': 1.3334,
   'normalized_risk': 49.995,
   'bias_level': 'High bias'},
  'ethnicity': {'accuracy': 0.6667,
   'proportion': 0.5,
   'rate': 1.3334,
   'normalized_risk': 49.995,
   'bias_level': 'High bias'},
  'age': {'accuracy': 0.4916,
   'proportion': 0.5,
   'rate': 0.9832,
   'normalized_risk': 100,
   'bias_level': 'Low/no bias'},
  'sex_ethnicity': {'accuracy': 0.6667,
   'proportion': 0.5,
   'rate': 1.3334,
   'normalized_risk': 49.995,
   'bias_level': 'High bias'}}}